stations_with_descriptionsThe manual: http://data.atoc.org/sites/all/themes/atoc/files/RSPS5045.pdf

The data: http://data.atoc.org/fares-data

In [1]:
import pandas as pd
import sys
sys.path.append("./..")
from importlib import reload

from src import data_prep
import config.indices_of_records as dicts

In [2]:
reload(dicts)
files_pattern = "RJFAF689"

### Flow file

In [44]:
reload(data_prep)

<module 'src.data_prep' from './..\\src\\data_prep.py'>

In [45]:
merged_data = data_prep.prep_fares_table(files_pattern=files_pattern)

In [46]:
merged_data.head()

,records_x,update_marker,record_type_x,origin_code,destination_code,route_code,status_code,usage_code,direction,end_date,...,toc,cross_london_ind,non_standard_disc_ind,publication_ind,flow_id,records_y,record_type_y,ticket_code,fare,restriction_code
0,RF0027003201000000AS3112299902012020ATO01Y0000049,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000049,RT00000497TF00005420,T,7TF,00005420,
1,RF0027003201000000AS3112299902012020ATO01Y0000049,R,F,0027,0032,01000,000,A,S,31122999,...,ATO,0,1,Y,0000049,RT00000497TS00003610,T,7TS,00003610,
2,RF0027003301000000AS3112299902012020ATO01Y0000028,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000028,RT00000287TF00006360,T,7TF,00006360,
3,RF0027003301000000AS3112299902012020ATO01Y0000028,R,F,0027,0033,01000,000,A,S,31122999,...,ATO,0,1,Y,0000028,RT00000287TS00004240,T,7TS,00004240,
4,RF0027003401000000AS3112299902012020ATO01Y0000165,R,F,0027,0034,01000,000,A,S,31122999,...,ATO,0,1,Y,0000165,RT00001657TF00007790,T,7TF,00007790,


## Locations file
Locations, including group locations

In [47]:
locations_records, synonym_records = data_prep.prep_locations_tables(files_pattern=files_pattern)

### Routes

In [48]:
routes_records, route_excl_records = data_prep.prep_routes_tables(files_pattern=files_pattern)

In [49]:
routes_records.head()

,records,update_marker,record_type,route_code,end_date,start_date,quote_date,description
1215,RR00000311229990303201807042017ANY PERMITTED ...,R,R,00000,31122999,03032018,07042017,ANY PERMITTED
1216,RR00002311229992306201107042017NOT OTHER ZONES...,R,R,00002,31122999,23062011,07042017,NOT OTHER ZONES
1217,RR00003060520200410201806052020STRATFORD/LONDO...,R,R,00003,06052020,04102018,06052020,STRATFORD/LONDON
1218,RR00003311229990705202006052020STRATFORD/LONDO...,R,R,00003,31122999,07052020,06052020,STRATFORD/LONDON
1219,RR00004311229992212201821122018VIA BLACKHORS R...,R,R,00004,31122999,22122018,21122018,VIA BLACKHORS RD


### Combining the data sets

In [50]:
routes_with_fares_filtered = data_prep.combine_fares_and_routes(
    merged_data=merged_data, routes_records=routes_records)

In [51]:
routes_fares_descriptions = routes_with_fares_filtered.merge(
    locations_records[["nlc_code", "description"]].drop_duplicates(),
    left_on="origin_code", right_on="nlc_code", how="left"
    )

In [67]:
routes_fares_descriptions

,update_marker,origin_code,destination_code,route_code,status_code,usage_code,direction,toc,cross_london_ind,non_standard_disc_ind,...,flow_id,ticket_code,fare,restriction_code,quote_date,route_description,nlc_code_origin,origin_station,nlc_code_destination,description
0,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TF,00005420,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
1,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TF,00005420,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
2,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TS,00003610,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
3,R,0027,0032,01000,000,A,S,ATO,0,1,...,0000049,7TS,00003610,,22062020,.,0027,TRVCARD 1,0032,LONDON ZONES 1-2
4,R,0027,0033,01000,000,A,S,ATO,0,1,...,0000028,7TF,00006360,,22062020,.,0027,TRVCARD 1,0033,LONDON ZONES 1-3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6195799,R,TRN1,3081,00000,000,G,S,IEC,1,1,...,0603700,SOR,00037300,,07042017,ANY PERMITTED,NaN,NaN,3081,HEATHROW RAIL
6195800,R,TRN1,3081,00000,000,G,S,IEC,1,1,...,0603700,SOS,00019000,,07042017,ANY PERMITTED,NaN,NaN,3081,HEATHROW RAIL
6195801,R,TRN1,3081,00000,000,G,S,IEC,1,1,...,0603700,SSR,00019160,1L,07042017,ANY PERMITTED,NaN,NaN,3081,HEATHROW RAIL
6195802,R,TRN1,3081,00000,000,G,S,IEC,1,1,...,0603700,SSS,00017260,1L,07042017,ANY PERMITTED,NaN,NaN,3081,HEATHROW RAIL


In [52]:
routes_fares_descriptions = data_prep.combine_fares_routes_locations(
    routes_with_fares_filtered=routes_with_fares_filtered,
    locations_records=locations_records)

6184580
6195804


### Ticket types

In [53]:
ticket_types = data_prep.prep_single_record_type_in_file_table(
    files_pattern=files_pattern,  dictionary_types=dicts.dict_ticket_types, file_ending="TTY")

In [54]:
ticket_types.head()

,records,update_marker,ticket_code,end_date,start_date,quote_date,description,tkt_class,tkt_type,tkt_group,...,max_children,min_children,restricted_by_date,restricted_by_train,restricted_by_area,validity_code,atb_description,lul_xlondon_issue,fare_multiplier,discount_category
0,R0AA311229991707201916072019SMART TKS 2SS...,R,0AA,31122999,17072019,16072019,SMART TKS,2,S,S,...,001,000,N,N,Y,87,SMART TKS,1,001,20
1,R0AB311229991707201916072019SMART TKR 2RS...,R,0AB,31122999,17072019,16072019,SMART TKR,2,R,S,...,001,000,N,N,Y,84,SMART TKR,2,001,20
2,R0AC311229992805201725052017SMART FDR 1RF...,R,0AC,31122999,28052017,25052017,SMART FDR,1,R,F,...,001,000,N,N,N,81,SMART FDR,2,001,01
3,R0AD311229992805201725052017SMART FDS 1SF...,R,0AD,31122999,28052017,25052017,SMART FDS,1,S,F,...,001,000,N,N,N,87,SMART FDS,1,001,01
4,R0AE311229992805201725052017SMART SDR 2RS...,R,0AE,31122999,28052017,25052017,SMART SDR,2,R,S,...,001,000,N,N,N,81,SMART SDR,2,001,01


### Ticket validity

In [55]:
validity_cols = ["validity_code", "start_date", "end_date", "description"]

In [56]:
ticket_validity = data_prep.prep_single_record_type_in_file_table(
    files_pattern=files_pattern, dictionary_types=dicts.dict_ticket_validity, file_ending="TVL")
ticket_validity[validity_cols].head()

,validity_code,start_date,end_date,description
0,00,01031991,31122999,(USE SEASON)
1,01,01031991,31122999,THREE DAYS
2,02,01031991,31122999,THREE MONTHS
3,03,01031991,31122999,ONE MONTH
4,04,01031991,31122999,AS ADVERTISED


### Combine ticket types and ticket validity

In [57]:
ticket_types_columns = ["ticket_code", "description", 'tkt_class', 'tkt_type', 'tkt_group',
                   'restricted_by_train', 'restricted_by_area', 'validity_code']

In [58]:
tickets = ticket_types[ticket_types_columns].merge(
    ticket_validity[validity_cols], on=["validity_code"],
    how="left", suffixes=["", "_validity"])

In [59]:
tickets.head()

,ticket_code,description,tkt_class,tkt_type,tkt_group,restricted_by_train,restricted_by_area,validity_code,start_date,end_date,description_validity
0,0AA,SMART TKS,2,S,S,N,Y,87,19122000,31122999,ON DATE SHOWN
1,0AB,SMART TKR,2,R,S,N,Y,84,19122000,31122999,SEE RESTRICTNS
2,0AC,SMART FDR,1,R,F,N,N,81,19122000,31122999,ON DATE SHOWN
3,0AD,SMART FDS,1,S,F,N,N,87,19122000,31122999,ON DATE SHOWN
4,0AE,SMART SDR,2,R,S,N,N,81,19122000,31122999,ON DATE SHOWN


In [60]:
routes_columns = ["route_code", "status_code", "usage_code", "direction", "toc",
                  "cross_london_ind", "non_standard_disc_ind", "ticket_code", "fare",
                  "restriction_code", "route_description", "origin_station", "description"]

In [61]:
routes_fares_descriptions_thin = routes_fares_descriptions[routes_columns]

In [62]:
final = routes_fares_descriptions_thin.merge(tickets, on="ticket_code", how="left", suffixes=["", "_ticket"])

In [63]:
final.head()

,route_code,status_code,usage_code,direction,toc,cross_london_ind,non_standard_disc_ind,ticket_code,fare,restriction_code,...,description_ticket,tkt_class,tkt_type,tkt_group,restricted_by_train,restricted_by_area,validity_code,start_date,end_date,description_validity
0,01000,000,A,S,ATO,0,1,7TF,00005420,,...,TRAVELCARD 7DF,1,N,F,N,N,00,01031991,31122999,(USE SEASON)
1,01000,000,A,S,ATO,0,1,7TF,00005420,,...,TRAVELCARD 7DF,1,N,F,N,N,00,01031991,31122999,(USE SEASON)
2,01000,000,A,S,ATO,0,1,7TF,00005420,,...,TRAVELCARD 7DF,1,N,F,N,N,00,01031991,31122999,(USE SEASON)
3,01000,000,A,S,ATO,0,1,7TF,00005420,,...,TRAVELCARD 7DF,1,N,F,N,N,00,01031991,31122999,(USE SEASON)
4,01000,000,A,S,ATO,0,1,7TS,00003610,,...,TRAVELCARD 7DS,2,N,S,N,N,00,01031991,31122999,(USE SEASON)


In [28]:
final.to_feather("../data/prepped_data")

In [29]:
data_in = pd.read_feather("../data/prepped_data")

Obviously does not have all the fares. I need to figure out the rules for calculating the fares. 
Maybe I could machine learn it :D

In [4]:
reload(dicts)

<module 'config.indices_of_records' from './..\\config\\indices_of_records.py'>

### Check non-derivable fares

In [8]:
non_derivable_fares = data_prep.prep_single_record_type_in_file_table(
    files_pattern=files_pattern, dictionary_types=dicts.non_derivable_fares, file_ending="NFO")

In [18]:
non_derivable_fares_with_locs = (
    non_derivable_fares
    .merge(locations_records[["nlc_code", "description"]].drop_duplicates(), 
           left_on="origin_code", right_on="nlc_code"))
non_derivable_fares_with_locs.rename(columns={"description":"origin"}, inplace=True)

non_derivable_fares_with_locs = (
    non_derivable_fares_with_locs
    .merge(locations_records[["nlc_code", "description"]].drop_duplicates(), 
           left_on="destination_code", right_on="nlc_code"))

non_derivable_fares_with_locs.rename(columns={"description":"destination"}, inplace=True)
non_derivable_fares_with_locs.drop(["nlc_code_x", "nlc_code_y"], axis=1, inplace=True)

In [101]:
non_derivable_fares_with_locs[non_derivable_fares_with_locs["origin"].str.contains("MANCH")].sort_values("destination").head()

,records,origin_code,destination_code,route_code,origin,destination
384152,R0438180500000 GSKO311229992205202022052020N...,0438,1805,00000,MANCHESTER STNS,ALFRETON
384270,R2950180500000 GSKO311229992205202022052020N...,2950,1805,00000,ECCLES (MANCHR),ALFRETON
384269,R2950180500000 GSKO200520201705202027102016N...,2950,1805,00000,ECCLES (MANCHR),ALFRETON
384220,R2800180500000 GSKO311229992205202022052020N...,2800,1805,00000,SWINTON MANCHR,ALFRETON
384219,R2800180500000 GSKO200520201705202027102016N...,2800,1805,00000,SWINTON MANCHR,ALFRETON


### Check non-standard discounts

In [79]:
non_standard_discounts = data_prep.prep_single_record_type_in_file_table(
    files_pattern=files_pattern, dictionary_types=dicts.non_derivable_fares, file_ending="FNS")

In [84]:
non_standard_discounts

,records,origin_code,destination_code,route_code
0,R****0032***** 7TF31122999070220020704201700...,****,0032,*****
1,R****0032***** 7TS31122999070220020704201700...,****,0032,*****
2,R****0032***** ADT31122999070220020704201700...,****,0032,*****
3,R****0032***** ODT31122999070220020704201700...,****,0032,*****
4,R****0032***** TRF31122999240920130704201700...,****,0032,*****
...,...,...,...,...
91983,RN770****01004 ECR311229991512201007042017N7...,N770,****,01004
91984,RN770****01004 ECS311229992801201107042017N7...,N770,****,01004
91985,RN770****01004******311229991512201007042017 ...,N770,****,01004
91986,RN770****01004***ECR311229991512201007042017N7...,N770,****,01004


In [95]:
non_standard_discounts_with_locs = (
    non_standard_discounts
    .merge(locations_records[["nlc_code", "description"]].drop_duplicates(), 
           left_on="origin_code", right_on="nlc_code", how="left"))
non_standard_discounts_with_locs.rename(columns={"description":"origin"}, inplace=True)

non_standard_discounts_with_locs = (
    non_standard_discounts_with_locs
    .merge(locations_records[["nlc_code", "description"]].drop_duplicates(), 
           left_on="destination_code", right_on="nlc_code", how="left"))

non_standard_discounts_with_locs.rename(columns={"description":"destination"}, inplace=True)
non_standard_discounts_with_locs.drop(["nlc_code_x", "nlc_code_y"], axis=1, inplace=True)

non_standard_discounts_with_locs.loc[pd.isna(non_standard_discounts_with_locs["destination"]), "destination"] = "0"
non_standard_discounts_with_locs.loc[pd.isna(non_standard_discounts_with_locs["origin"]), "origin"] = "0"

In [102]:
non_standard_discounts_with_locs

,records,origin_code,destination_code,route_code,origin,destination
0,R****0032***** 7TF31122999070220020704201700...,****,0032,*****,0,LONDON ZONES 1-2
1,R****0032***** 7TS31122999070220020704201700...,****,0032,*****,0,LONDON ZONES 1-2
2,R****0032***** ADT31122999070220020704201700...,****,0032,*****,0,LONDON ZONES 1-2
3,R****0032***** ODT31122999070220020704201700...,****,0032,*****,0,LONDON ZONES 1-2
4,R****0032***** TRF31122999240920130704201700...,****,0032,*****,0,LONDON ZONES 1-2
...,...,...,...,...,...,...
92978,RN770****01004 ECR311229991512201007042017N7...,N770,****,01004,UNIVERSITY NI,0
92979,RN770****01004 ECS311229992801201107042017N7...,N770,****,01004,UNIVERSITY NI,0
92980,RN770****01004******311229991512201007042017 ...,N770,****,01004,UNIVERSITY NI,0
92981,RN770****01004***ECR311229991512201007042017N7...,N770,****,01004,UNIVERSITY NI,0


In [103]:
non_standard_discounts_with_locs.loc[non_standard_discounts_with_locs["destination"].str.contains("MANCH")].sort_values("destination").head()

,records,origin_code,destination_code,route_code,origin,destination
8304,R****296100318JCP7DS31122999130720171207201729...,****,2961,00318,0,MANCHESTER AIRPT
8283,R****296100318 7DS31122999130820100704201729...,****,2961,00318,0,MANCHESTER AIRPT
8284,R****296100318 CDR31122999110920010704201729...,****,2961,00318,0,MANCHESTER AIRPT
8285,R****296100318 CDS31122999110920010704201729...,****,2961,00318,0,MANCHESTER AIRPT
8286,R****296100318 FDS31122999200420110704201729...,****,2961,00318,0,MANCHESTER AIRPT


### Explore rules

In [74]:
explore = final.copy()
explore.loc[pd.isna(explore["origin_station"]), "origin_station"] = "0"

In [78]:
explore.loc[explore["origin_station"].str.contains("MANCH"), ["description", "description_ticket"]]

,description,description_ticket
143636,LONDON ZONES 1-6,SAT DYTCRD
143637,LONDON ZONES 1-6,SAT DYTCRD
143638,LONDON ZONES 1-6,SAT DYTCRD
143639,ZONE U1245 LONDN,ANYTIME 1R
143640,ZONE U1245 LONDN,ANYTIME 1S
...,...,...
1043417,WAKEFIELD WESTGT,ADVANCE
1043418,WAKEFIELD WESTGT,SALE ADVANCE
1043419,WAKEFIELD WESTGT,ADVANCE
1043420,WAKEFIELD WESTGT,SALE ADVANCE
